In [35]:
import pandas as pd
import numpy as np

In [36]:
import json
import os

# Define o caminho do arquivo JSON
filepath = './dados.json'
dados = None

# Verifica se o arquivo existe antes de tentar lê-lo
if os.path.exists(filepath):
    try:
        # Abre o arquivo em modo de leitura ('r')
        with open(filepath, 'r', encoding='utf-8') as f:
            # Carrega o conteúdo do arquivo JSON para um dicionário Python
            dados = json.load(f)


    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar o JSON: {e}")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
else:
    print(f"Erro: O arquivo '{filepath}' não foi encontrado.")

In [37]:
len(dados)

1710

In [38]:
df = pd.json_normalize(dados)


In [39]:
df

,valor,id,timestamp,sensor.id,sensor.nome
0,99.038863,1710,2025-06-11T01:48:58.929921,5,sensor_k
1,42.299256,1709,2025-06-11T01:48:58.919727,4,sensor_p
2,13.062344,1708,2025-06-11T01:48:58.909831,3,sensor_ph
3,21.936602,1707,2025-06-11T01:48:58.882791,2,sensor_humidity
4,87.080089,1706,2025-06-11T01:48:58.865412,1,sensor_temperature
...,...,...,...,...,...
1705,23.764496,5,2025-05-20T00:27:44.875620,5,sensor_k
1706,19.326224,4,2025-05-20T00:27:44.871385,4,sensor_p
1707,5.198631,3,2025-05-20T00:27:44.866688,3,sensor_ph
1708,24.014174,2,2025-05-20T00:27:44.861143,2,sensor_humidity


In [40]:
df.to_csv("./dados.csv")

In [41]:
# Criar o DataFrame inicial
df = df.rename(columns={'id': 'leitura_id', 'sensor.id': 'sensor_id', 'sensor.nome': 'sensor_nome'})

# 2. Transformar o DataFrame
# Converter a coluna de timestamp para o formato de data/hora
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Pivotar a tabela para que os sensores virem colunas
df_pivot = df.pivot_table(index='timestamp', columns='sensor_nome', values='valor')

# Agrupar as leituras por segundo (pegando a última de cada segundo) e preencher valores ausentes
df_wide = df_pivot.resample('1S').last().ffill().bfill()

# 3. Aplicar a lógica para criar a coluna de status
# Definir as condições com base nos comentários do seu código
condicoes = (
    (df_wide['sensor_ph'] > 7) &
    (df_wide['sensor_temperature'] > 30) &
    (df_wide['sensor_humidity'] < 50)
)

# Criar a coluna 'status': 1 para HIGH (condição verdadeira), 0 para LOW (falsa)
df_wide['target'] = np.where(condicoes, 1, 0)

# Criar a coluna 'motivo'
motivo_texto = "pH > 7, Temperatura > 30°C e Umidade < 50%"
df_wide['motivo'] = np.where(condicoes, motivo_texto, "")


# 4. Exibir o resultado e salvar em um novo CSV
print("DataFrame com Status e Motivo:")
print(df_wide.head())
# save balanced dataset
df_wide = df_wide[1000000:]
# Salvar o novo dataframe em um arquivo CSV
output_filename = 'dados_sensores_com_status.csv'
df_wide.to_csv(output_filename, sep=',', encoding='utf-8-sig')

print(f"\nArquivo '{output_filename}' salvo com sucesso!")

DataFrame com Status e Motivo:
sensor_nome          sensor_humidity   sensor_k   sensor_p  sensor_ph  \
timestamp                                                               
2025-05-20 00:27:44        24.014174  23.764496  19.326224   5.198631   
2025-05-20 00:27:45        11.102870  96.249995  19.184553   7.271486   
2025-05-20 00:27:46        76.252779   6.311321  69.251505   6.438823   
2025-05-20 00:27:47        65.164521   6.311321  69.251505   6.438823   
2025-05-20 00:27:48        65.164521   6.109431  40.865383   5.432344   

sensor_nome          sensor_temperature  target  \
timestamp                                         
2025-05-20 00:27:44           68.215034       0   
2025-05-20 00:27:45           60.135394       1   
2025-05-20 00:27:46           58.053577       0   
2025-05-20 00:27:47           22.088025       0   
2025-05-20 00:27:48           22.088025       0   

sensor_nome                                              motivo  
timestamp                        

In [44]:
df_wide["target"].value_counts()

target
1    900978
0      4697
Name: count, dtype: int64